In [8]:
from RNARepLearn.datasets import CombinedRfamDataset
from RNARepLearn.utils import train_val_test_loaders, add_backbone, generate_edges,reconstruct_bpp
from RNARepLearn.train import MaskedTraining
from RNARepLearn.models import Encoder_Decoder_Model, Seq_Struc_GNN, CombinedGNN
from RNARepLearn.layers import Sep_Seq_Struc_Layer
from RNARepLearn.modules import CNN_Seq, RPINetEncoder, TE_Decoder, AttentionDecoder
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, TransformerConv
from torch_geometric.transforms import AddLaplacianEigenvectorPE
from torch_geometric.utils import to_dense_batch
import numpy as np

In [2]:
from RNARepLearn.datasets import CombinedRfamDataset
from RNARepLearn.utils import train_val_test_loaders, add_backbone

ds = CombinedRfamDataset("/lustre/groups/crna01/workspace/nicolas_msc/RNARepLearn/rfam/data/raw/processed/release-14.8", None ,"All_300")# , add_LPE=True, add_backbone=True, transform=AddLaplacianEigenvectorPE(16, attr_name=None)
batch_size = 64

Processing...


Processing complete


Done!


In [3]:
train, val, test = train_val_test_loaders(ds, 0.001, 0.001, 0.998, 64, num_workers = 4)

Training:	695
Test:	694216
Validation:	697


In [4]:
gnn = Seq_Struc_GNN(4, 128, 6, GCNConv)
#gnn = CombinedGNN(20,128, 4, TransformerConv, edge_dim=2).double()

#tconv = TransformerConv(20, 128, edge_dim=2)
#gnn = RPINetEncoder(4,64,3,9)
#decoder = TE_Decoder(batch_size,128)
decoder = AttentionDecoder(128, 4)

In [5]:
model = Encoder_Decoder_Model(gnn, decoder)

In [39]:
batch = next(iter(train))

In [40]:


pred_nucs, pred_bpp = model(batch)

pred_nucs = pred_nucs
pred_bpp = pred_bpp



In [42]:
true_bpp = torch.tensor(reconstruct_bpp(batch.edge_index, batch.edge_weight, (len(batch.x),len(batch.x))))
true_mat = true_bpp!=0
pred_mat = pred_bpp!=0
all_mat = torch.ones(len(true_mat)**2, dtype=torch.bool).reshape(len(true_mat),len(true_mat))

In [47]:
TP = torch.sum(torch.logical_and(true_mat, pred_mat)) #intersection true and pred
FP = torch.sum(torch.logical_and(torch.logical_and(all_mat, ~true_mat), pred_mat)) #setdiff all_edges / true_edges (e.g. all_edges and !ture_edges)
FN = torch.sum(torch.logical_and(torch.logical_and(all_mat, ~pred_mat), true_mat))

In [51]:
TP

tensor(402997)

In [30]:
all_edges = generate_edges(torch.ones(len(pred_nucs),len(pred_nucs))).contiguous().numpy()
pred_edges = generate_edges(pred_bpp).contiguous().numpy()
true_edges = batch.edge_index.T.contiguous().numpy()

dtype={'names':['f{}'.format(i) for i in range(pred_edges.shape[1])],
       'formats':pred_edges.shape[1] * [pred_edges.dtype]}

true_pos = len(np.intersect1d(pred_edges.view(dtype), true_edges.view(dtype), assume_unique=True))

false_pos = len(np.intersect1d(np.setdiff1d(all_edges.view(dtype), true_edges.view(dtype), assume_unique=True), pred_edges.view(dtype), assume_unique=True))

false_neg = len(np.intersect1d(true_edges,np.setdiff1d(all_edges,pred_edges, assume_unique=True),assume_unique=True))

precision = true_pos/(true_pos+false_pos)

recall = true_pos/(true_pos+false_neg)

f1 = 2 * (precision*recall)/(precision+recall)

In [27]:
true_pos.view(pred_edges.dtype).reshape(-1, pred_edges.shape[1]).shape

(396664, 2)

In [19]:
def tuple_func(x):
    return tuple(x)
func = np.vectorize(tuple_func)

In [22]:
np.in1d(all_edges, pred_edges).shape

(111601800,)

(55800900, 2)

In [18]:
all_edges = [tuple(i) for i in all_edges]
pred_edges = [tuple()]

In [21]:
import numpy as np

In [43]:
A = np.array([[1,4],[2,5],[4,1]])
B = np.array([[1,4],[3,6],[7,8], [4,1]])

nrows, ncols = A.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [A.dtype]}

C = np.intersect1d(A.view(dtype), B.view(dtype))

# This last bit is optional if you're okay with "C" being a structured array...
C = C.view(A.dtype).reshape(-1, ncols)

In [47]:
pred_edges.shape

(45819361, 2)

In [27]:
training = MaskedTraining(model, 1, 15, None, print_out=True)

Creating new SummaryWriter


2023-03-06 19:55:57.079212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [26]:
training.run(train, val)

Training running on device: cpu
[Epoch    1/   1] [Batch   10/  11] Loss:  9.76e+00 Nucleotide-Loss:  1.39e+00 Edge-Loss:  8.38e+00  Memory    0
[Epoch    1/   1] Epoch-Loss:  9.76e+00 Epoch-Loss Val:  9.79e+00 Nucleotide-Loss Val:  1.39e+00 Edge-Loss Val:  8.41e+00  Memory    0


In [27]:
batch = next(iter(train))